In [1]:
import pandas as pd

In [2]:
year = 2009
election_round = 2 # Values accepted are {1, 2}

In [3]:
df = pd.read_csv(f"data/Romania/{year}_votes.csv")
df["polling_id"] = df.apply(lambda x: f"{x.election_id}_{x.county_id}_{x.polling_id}", axis=1)
df = df[df["NumarTur"] == election_round]
df.head()

,Candidat,AfilierePolitica,Voturi,Procent,NumarTur,polling_id,county_id,election_id
0,Traian Băsescu,PDL,493,53.64,2,261_1_1,1,261
1,Mircea Geoană,PSD + PC,426,46.35,2,261_1_1,1,261
12,Traian Băsescu,PDL,412,57.54,2,261_1_2,1,261
13,Mircea Geoană,PSD + PC,304,42.45,2,261_1_2,1,261
25,Traian Băsescu,PDL,491,55.41,2,261_1_3,1,261


In [4]:
df["rank"] = df.groupby(["polling_id"])["Voturi"].rank("min", ascending=False).astype(int)

In [5]:
tt = df.groupby(["polling_id", "Candidat"]).agg({"Voturi": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "Candidat", "rate"]]

In [6]:
df = pd.merge(df, tt, on=["polling_id", "Candidat"])

In [7]:
df_filtered = df[["polling_id", "Candidat", "Voturi", "rank", "rate"]]\
    .rename(columns={"Candidat": "candidate", "Voturi": "value"})

df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

In [8]:
e_round = ["first_round", "runoff"]
df_filtered.to_csv(f"data_output/Romania/{year}_{e_round[election_round - 1]}.csv.gzip", compression="gzip", index=False)

In [14]:
df_location = pd.read_csv(f"data/Romania/{year}_polling_stations.csv")
df_location.head()

,SIRUTA,Nume,Id,county_id,NSV,Adresa,location_id,COD_JUD,DEN_JUD,election_id
0,87312,ALBAC,87312,1,193,"Caminul cultural Albac, strada Closca, nr. 9, ...",87312,1,Alba,261
1,87312,ALBAC,87312,1,194,"Scoala cu clasele I - IV Cionesti, sat. Ciones...",87312,1,Alba,261
2,87312,ALBAC,87312,1,195,"Scoala cu clasele I - IV Costesti, sat. Costes...",87312,1,Alba,261
3,87312,ALBAC,87312,1,196,"Scoala cu clasele I - IV Rogoz, sat. Rogoz, Sa...",87312,1,Alba,261
4,87312,ALBAC,87312,1,197,"Scoala cu clasele I - IV Potionci, sat. Potion...",87312,1,Alba,261


In [15]:
df_location["polling_id"] = df_location.apply(lambda x: f"{x.election_id}_{x.COD_JUD}_{x.NSV}", axis=1)
df_location = df_location.rename(columns={
    "SIRUTA": "location_id",
    "NSV": "polling_place",
    "Nume": "location_id",
    "DEN_JUD": "county_name",
    "COD_JUD": "county_id",
    "Adresa": "address"
})

df_location.to_csv(f"data_output/Romania/{year}_{e_round[election_round - 1]}_location.csv.gzip", compression="gzip", index=False)
df_location

,location_id,location_id,Id,county_id,polling_place,address,location_id,county_id,county_name,election_id,polling_id
0,87312,ALBAC,87312,1,193,"Caminul cultural Albac, strada Closca, nr. 9, ...",87312,1,Alba,261,261_1_193
1,87312,ALBAC,87312,1,194,"Scoala cu clasele I - IV Cionesti, sat. Ciones...",87312,1,Alba,261,261_1_194
2,87312,ALBAC,87312,1,195,"Scoala cu clasele I - IV Costesti, sat. Costes...",87312,1,Alba,261,261_1_195
3,87312,ALBAC,87312,1,196,"Scoala cu clasele I - IV Rogoz, sat. Rogoz, Sa...",87312,1,Alba,261,261_1_196
4,87312,ALBAC,87312,1,197,"Scoala cu clasele I - IV Potionci, sat. Potion...",87312,1,Alba,261,261_1_197
...,...,...,...,...,...,...,...,...,...,...,...
21701,90416,VULTURU,90416,41,426,BOTIRLAU SCOALA GENERALA,90416,41,Vrancea,261,261_41_426
21702,90416,VULTURU,90416,41,427,HINGULESTI SCOALA GENERALA,90416,41,Vrancea,261,261_41_427
21703,90416,VULTURU,90416,41,428,MALURI SCOALA,90416,41,Vrancea,261,261_41_428
21704,90416,VULTURU,90416,41,429,VADU ROSCA SCOALA,90416,41,Vrancea,261,261_41_429
